In [ ]:
#Agrego columna con leads antes de conversion
temp = dfData.sort_values('timestamp').groupby('person')
lDictLeads = []
for name, group in temp:
    leads = {}
    for row in group.itertuples():
        model = row.model
        mes = int(row.timestamp[6])
        dia = int(row.timestamp[8:10])
        if row.event == 'lead' and mes >= 5 and dia >= 20:
            if model in leads:
                leads[model] += 1
            else:
                leads[model] = 1
        elif row.event == 'conversion' and (model in leads):
            if leads[model] > 1:
                leads[model] -= 1
            else:
                del leads[model]
    if leads.values():
        leads = 1
    else:
        leads = 0
    lDictLeads.append({'person' : name, 'leads' : leads})
dfLeads = pd.DataFrame(lDictLeads)

In [ ]:
#Feature channels
dfT = dfData[['person', 'channel']].dropna()
dfT = dfT.loc[dfT['channel'] != 'Unknown']
dfT = dfT.loc[dfT['channel'] != 'Email']
dfT = dfT.loc[dfT['channel'] != 'Social']
dfQChannels = dfT.groupby('person')['channel'].count().to_frame()
dfPE = pd.get_dummies(dfT, columns=['channel']).groupby('person').sum()
columnasCanales = dfPE.columns.values
dfPE = dfPE.merge(dfQChannels, on='person')
for canalColumna in columnasCanales:
    dfPE[canalColumna] /= dfPE['channel']
dfPE = dfPE.drop('channel', axis=1)
dfUsers = dfUsers.merge(dfPE, on='person', how='left').fillna(0)

In [ ]:
#Feature new vs returning
def f(s):
    if s == 'Returning':
        return 1
    return 0

dfPE = dfData.dropna(subset=['new_vs_returning']).groupby('person')['new_vs_returning'].max().to_frame()
dfPE['newOrReturning'] = dfPE['new_vs_returning'].apply(f)
dfUsers = dfUsers.merge(dfPE.drop('new_vs_returning', axis=1), on='person', how='left').fillna(0)

In [ ]:
#Feature screen resolution
dfPE = dfData.dropna(subset=['screen_resolution'])
dfPE['screenSize'] = dfPE['screen_resolution'].apply(lambda x : (int(str(x).split('x')[0]) + int(str(x).split('x')[1])) // 2)
dfPE = dfPE.groupby('person')['screenSize'].max().to_frame()
dfUsers = dfUsers.merge(dfPE, on='person', how='left').fillna(0)